<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_vscode_branch/HW3/HW03_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_vscode_branch/HW2/ML2021HW2_modified.ipynb" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# MEGA
# !sudo apt install megatools
# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"

# Unzip the dataset.
# This may take some time.
!unzip -q food-11.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
To: /content/food-11.zip
100% 963M/963M [00:16<00:00, 57.3MB/s]


In [2]:
#import pytorch
import torch

# torch.backends.cudnn: set CNN algorithmtorch.backends.cudnn
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# get the current available device ('cpu' or 'cuda')
def get_device():
#    return 'cpu'
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(torch.cuda.is_available())
#set random variable
import numpy as np
myseed = 1
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

True


In [30]:
# import torch
# import torch.nn as nn
# m = nn.Conv2d(1, 1, 2 , stride=2, padding=(9,9),bias = False)
# test = torch.tensor([[[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
# print(test)
# print(m(test))

In [54]:
# class Label_Changable_Folder(DatasetFolder):
#     def __init__(self,root,loader,extensions,transform,target_transform,is_valid_file):
#             super().__init__(root, loader, extensions , transform, target_transform,is_valid_file)
#     def change_targets(self , index ,new_target):
#       self.targets[index] = new_target


In [21]:
from torchvision.transforms import v2
from torchvision.datasets import DatasetFolder
from PIL import Image
def get_tfm(mode,data_info):
    if mode in ["val","test"]:
      tfm = v2.Compose([
      v2.ToImage(),
      v2.ToDtype(torch.float32, scale=True),
      v2.Resize(data_info["size"]),
      ])
    elif mode in ["train_origin","train"]:
      tfm = v2.Compose([
      v2.ToImage(),
      v2.ToDtype(torch.float32, scale=True),
#      v2.RandomResizedCrop(size=data_info["size"],scale=(0.8, 1.0), antialias=True),
      v2.Resize(data_info["size"]),
      v2.RandomHorizontalFlip(p=0.5),
      ])
    return tfm
def food_f(mode,data_info):
    dataset = DatasetFolder(data_info[mode]["path"], \
          loader=lambda x: Image.open(x), \
          extensions="jpg", transform=get_tfm(mode,data_info))
    print('Size of {} data: {}'.format(mode,len(list(dataset))))
    return dataset
#create a dict of functions and path w.r.t. different mode
data_info = {
    "train_origin":{"path":"./food-11/training/labeled"},
    "train":{"path":"./food-11/training/unlabeled"},
    "val":{"path":"./food-11/validation"},
    "test":{"path":"./food-11/testing"},
    "size" :(128,128)
}

In [33]:
# tfm = v2.Compose([
#       v2.ToImage(),
#       v2.ToDtype(torch.float32, scale=True),
#       v2.Resize(data_info["size"]),
#       ])
# print(type(tfm))

In [34]:
# x = 500# x = int(max(prep))
# print(list(food_f("val"))[x][1])

In [4]:
h_paras = {
    # maximum number of epochs
    'n_epochs': 2,
    # mini-batch size for dataloader
    'batch_size': 128,
    # optimization algorithm (optimizer in torch.optim)
    'optimizer': 'Adam',
    # hyper-parameters for the optimizer (depends on which optimizer you are using)
    'optim_hparas': {
        # learning rate of Adam
        'lr': 0.0003,
        "weight_decay" : 1e-5
    },
    # your model will be saved here
    'save_path': './model.pth',
    'early_stop': 10,
}

In [5]:
from torch.utils.data import DataLoader

# Construct data loaders.
train_set = DataLoader(food_f("train_origin",data_info), batch_size=h_paras["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
valid_set = DataLoader(food_f("val",data_info), batch_size=h_paras["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
test_set = DataLoader(food_f("test",data_info), batch_size=h_paras["batch_size"], shuffle=False)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Size of train_origin data: 3080
Size of val data: 660
Size of test data: 3347


In [6]:
for i , t in enumerate(valid_set):
    if i ==0:
        test_t = t[0]
#        print(type(t[0]),t[0].size())
        print(test_t.size())
        x = test_t[list(range(16))]
        print(x.size())

torch.Size([128, 3, 128, 128])
torch.Size([16, 3, 128, 128])


In [7]:
from torch import nn
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 32, 32]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.MaxPool2d(4, 4, 0),
        )
        self.flat_layer = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(int(256 * data_info["size"][0]/16 * data_info["size"][1]/16), 256),
            nn.LeakyReLU(),
            nn.Linear(256, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 11)
        )
        self.criterion = nn.CrossEntropyLoss()
    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = self.flat_layer(x)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x
    def cal_loss(self, pred, target):
        return self.criterion(pred, target)

In [ ]:
#%conda install ipywidgets

In [14]:
from torch.utils.data import Dataset
class Semi_Dataset_preparation(Dataset):
  def __init__(self,mode,data,label):
      self.mode = mode
      self.data = data
      self.label = label
      self.dim = self.data.shape[0]
      print('Finished reading the {mode} set of Dataset ({len} samples found, each dim = {dim})'
              .format(mode = self.mode, len =len(self.data), dim=self.dim))

  def __getitem__(self, index):
      return self.data[index], self.label[index]
  def __len__(self):
      # Returns the size of the dataset
      return len(self.data)

In [40]:
from tqdm.auto import tqdm
from torch.utils.data import ConcatDataset
def get_pseudo_labels(model, train_set , h_paras , threshold=0.65):

    softmax = nn.Softmax(dim=-1)
    model.eval()
    # Construct a data loader.
    train_unlabeled_set = \
    DataLoader(food_f("train",data_info), \
               batch_size=h_paras["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
    new_train_set = train_set
    datas = []
    labels = []
    for batch in tqdm(train_unlabeled_set):
        img = batch[0]

        with torch.no_grad():
            img_d = img.to(device)
            prep = model(img_d)

        predicts = softmax(prep)
        for  row_index , probs in enumerate(predicts):
          max_prob , max_column_index = probs.max(dim = -1)
          if max_prob.cpu() > threshold:
            datas.append(img[row_index].tolist())
            labels.append(max_column_index.tolist())
    if (len(datas))>0:
      add_set = Semi_Dataset_preparation("unlabel_training",torch.FloatTensor(datas),torch.LongTensor(labels))
      add_set_loader = DataLoader(add_set, batch_size=h_paras["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
      new_train_set = ConcatDataset([train_set , add_set_loader])
    else:
      new_train_set = train_set

    model.train()
    return new_train_set

In [11]:
# from tqdm.auto import tqdm
# from torch.utils.data import ConcatDataset
# def get_pseudo_labels(model, threshold=0.65):
#     # Make sure the model is in eval mode.
#     model.eval()
#     # Define softmax function.
#     softmax = nn.Softmax(dim=-1)

#     # Iterate over the dataset by batches.
#     datas = []
#     labels = []
#     for batch in tqdm(train_unlabeled_set):
#         img = batch[0]
#         # Forward the data
#         # Using torch.no_grad() accelerates the forward process.
#         with torch.no_grad():
#             img_d = img.to(device)
#             prep = model(img_d)

#         # Obtain the probability distributions by applying softmax on logits.
#         predicts = softmax(prep)
#         for  row_index , probs in enumerate(predicts):
#             max_prob , max_column_index = probs.max(dim = -1)
#             if max_prob > threshold:
#                 datas.append(img[row_index].tolist())
#                 labels += (max_column_index.tolist())
#     new_train_set = torch.FloatTensor(datas)
#     new_train_label = torch.LongTensor(labels)
#     model.train()
#     return new_train_set,new_train_label

In [19]:
def train(train_set,model_d,optimizer,device):
    # set model to training mode
    model_d.train()
    total_correct_number = 0
    train_loss_list = []
    # iterate through the dataloader
    for data , label in tqdm(train_set):
      # move data to device (cpu/cuda)
      data_d , label_d = data.to(device), label.to(device)
      # forward pass (compute output tensor)
      pred = (model_d(data_d))
      # get the index of the class with the highest probability
      max_prob_values, max_prob_indexs = torch.max(pred, dim = 1)
      correct_number = (max_prob_indexs.cpu() == label_d.cpu()).sum().item()
#      print(correct_number)
      total_correct_number += correct_number
      # compute loss
      loss = model_d.cal_loss(pred , label_d)
      # compute gradient (backpropagation)
      loss.backward()
      # Clip the gradient norms for stable training.
      nn.utils.clip_grad_norm_(model_d.parameters(), max_norm=10)
      # update model with optimizer
      optimizer.step()
      # set optimizer gradient to zero
      optimizer.zero_grad()
      train_loss_list.append(loss.detach().cpu().item())
    acc = total_correct_number/len(train_set.dataset)
#    print(acc , train_loss_list)
    return acc , train_loss_list

In [11]:
def val(val_set,model_d,device):
    # set model to evalutation mode
    model_d.eval()
    total_correct_number = 0
    total_loss = 0
    # iterate through the dataloader
    for data , label in tqdm(val_set):
    # move data to device (cpu/cuda)
      data_d, label_d = data.to(device), label.to(device)
      # disable gradient calculation
      with torch.no_grad():
        # forward pass (compute output)
        pred = model_d(data_d)
        # get the index of the class with the highest probability
        max_prob_values, max_prob_indexs = torch.max(pred, dim = 1)
        total_correct_number += (max_prob_indexs.cpu() == label_d.cpu()).sum().item()
        # compute loss
        mse_loss = model_d.cal_loss(pred, label_d)
      # accumulate loss
      batch_size = len(data_d)
      total_loss += mse_loss.detach().cpu().item() * batch_size

    # compute averaged loss
    totol_size = len(val_set.dataset)
    acc = total_correct_number/totol_size
    avg_loss =  total_loss/totol_size
    return acc, avg_loss

In [22]:
def train_val_process(training_set, validation_set, data_info, model, h_paras, device , do_semi = False):
    # Initialize a model, and put it on the device specified.
    model_d = model.to(device)
    model_d.device = device
    optimizer = torch.optim.Adam(model_d.parameters(), **h_paras["optim_hparas"])

    # The number of training epochs.
    n_epochs = h_paras["n_epochs"]

    # record training accuracy
    acc_record = {'train': [], "val": []}
    # record training loss
    loss_record = {'train': [], "val": []}

    # accuracy paras
    best_acc = 0
    # early-stoping paras
    early_stop_cnt = 0

    for epoch in range(n_epochs):
        # ---------- Training ----------
        train_acc , train_loss_list = train(training_set,model_d,optimizer,device)
        # save accuracy to acc_record['train']
        acc_record['train'].append(train_acc)
      # save loss to loss_record['train']
        loss_record['train'].append(train_loss_list)

        # ---------- Do semi-supervised learning ----------
        if do_semi == True:
          training_set = get_pseudo_labels(model_d, training_set,h_paras)

        # ---------- Validation ----------
        val_acc , val_loss = val(validation_set,model_d,device)
        # Print the information.
        acc_record["val"].append(val_acc)
        # save loss to loss_record["val"]
        loss_record["val"].append(val_loss)

        print(' model epoch = {:4d}, train_loss = {:.4f} , val_loss = {:.4f})'\
        .format(epoch+1 , train_loss_list[-1] , val_loss))
        print('train set accuracy = {:.3f}'.format(train_acc))

        # ---------- Early Stop ----------
        if val_acc > best_acc:
          best_acc = val_acc
          torch.save(model.state_dict(), h_paras["save_path"])
          print('saving model with acc {:.3f}'.format(best_acc))
          early_stop_cnt = 0
        else:
          early_stop_cnt += 1
        # Check early stop criteria
        if early_stop_cnt > h_paras['early_stop']:
            # Stop training if your model stops improving
            # for "h_paras['early_stop']" epochs.
            break

    return acc_record , loss_record


In [41]:
# Construct model
model = Classifier()
model_acc_record, model_loss_record = train_val_process(train_set, valid_set, data_info, model, h_paras, device, do_semi = True)

  0%|          | 0/25 [00:00<?, ?it/s]

Size of train data: 6786


  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

 model epoch =    1, train_loss = 2.7456 , val_loss = 2.7401)
train set accuracy = 0.181
saving model with acc 0.133


...

Size of train data: 6786


  0%|          | 0/54 [00:00<?, ?it/s]

Finished reading the unlabel_training set of Dataset (65 samples found, each dim = 65)


  0%|          | 0/6 [00:00<?, ?it/s]

 model epoch =    2, train_loss = 1.8454 , val_loss = 2.1878)
train set accuracy = 0.303
saving model with acc 0.215


In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_set):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        pred = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(pred.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")